# 📰 Web Scraping — Yahoo Finance
### สำหรับ NLP Term Project: Phase 1.1 (Data Collection)

**กลยุทธ์:** รวม 2 วิธีเข้าด้วยกัน
- **RSS Feed** → เสถียร 100%, ได้วันที่จริง, ได้ description
- **HTML Scraping** → ปริมาณเยอะกว่า, ครอบคลุมหลายหมวด

**เป้าหมาย:** ได้ข้อมูลข่าวการเงินปี 2025 อย่างน้อย **500-1,000 รายการ**

---
## 🔧 Step 0: ติดตั้งและ Import

In [1]:
!pip install requests beautifulsoup4 pandas lxml -q
print('✅ ติดตั้งสำเร็จ')

✅ ติดตั้งสำเร็จ


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import random
from datetime import datetime
import matplotlib.pyplot as plt

# Headers ที่ทำให้ดูเหมือน Browser จริง — สำคัญมาก ถ้าไม่ใส่ Yahoo จะ Block
HEADERS = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/120.0.0.0 Safari/537.36'
    ),
    'Accept':          'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection':      'keep-alive',
    'Referer':         'https://finance.yahoo.com/'
}

TODAY = datetime.now().strftime('%Y-%m-%d')
print(f'✅ Import สำเร็จ | วันที่รันโค้ด: {TODAY}')

✅ Import สำเร็จ | วันที่รันโค้ด: 2026-02-18


---
## 🟢 Step 1: RSS Feed Scraping (วิธีหลัก — เสถียรที่สุด)

In [3]:
# =====================================================
# RSS FEED SCRAPING
# ดึงผ่าน XML ที่ Yahoo เตรียมไว้ให้
# ข้อดี: ได้วันที่จริง, description, เสถียร 100%
# =====================================================

# รายการ RSS URL ครอบคลุมทุกหมวดการเงิน
RSS_FEEDS = {
    # ข่าวทั่วไป
    'top_stories':    'https://finance.yahoo.com/news/rssindex',
    'latest_news':    'https://finance.yahoo.com/rss/topstories',

    # ดัชนีตลาดสำคัญ
    'sp500':          'https://finance.yahoo.com/rss/headline?s=^GSPC',
    'nasdaq':         'https://finance.yahoo.com/rss/headline?s=^IXIC',
    'dowjones':       'https://finance.yahoo.com/rss/headline?s=^DJI',

    # หุ้น Tech
    'apple':          'https://finance.yahoo.com/rss/headline?s=AAPL',
    'microsoft':      'https://finance.yahoo.com/rss/headline?s=MSFT',
    'nvidia':         'https://finance.yahoo.com/rss/headline?s=NVDA',
    'google':         'https://finance.yahoo.com/rss/headline?s=GOOGL',
    'meta':           'https://finance.yahoo.com/rss/headline?s=META',
    'tesla':          'https://finance.yahoo.com/rss/headline?s=TSLA',
    'amazon':         'https://finance.yahoo.com/rss/headline?s=AMZN',

    # หุ้น Banking
    'jpmorgan':       'https://finance.yahoo.com/rss/headline?s=JPM',
    'bank_america':   'https://finance.yahoo.com/rss/headline?s=BAC',
    'goldman':        'https://finance.yahoo.com/rss/headline?s=GS',

    # สินทรัพย์อื่น
    'bitcoin':        'https://finance.yahoo.com/rss/headline?s=BTC-USD',
    'ethereum':       'https://finance.yahoo.com/rss/headline?s=ETH-USD',
    'gold':           'https://finance.yahoo.com/rss/headline?s=GC=F',
    'oil':            'https://finance.yahoo.com/rss/headline?s=CL=F',
}


def scrape_rss(feed_url, category_name):
    """
    ดึงข่าวจาก Yahoo Finance RSS Feed
    คืนค่า: list of dict พร้อม headline, date จริง, description
    """
    try:
        time.sleep(random.uniform(1.0, 2.0))  # หน่วงเวลาสุ่มป้องกัน Block
        response = requests.get(feed_url, headers=HEADERS, timeout=10)

        if response.status_code != 200:
            return []

        # ใช้ xml parser เพราะ RSS เป็น XML ไม่ใช่ HTML
        soup  = BeautifulSoup(response.text, 'xml')
        items = soup.find_all('item')
        results = []

        for item in items:
            title    = item.find('title')
            pub_date = item.find('pubDate')
            link     = item.find('link')
            desc     = item.find('description')

            if not title:
                continue

            headline_text = title.get_text(strip=True)
            if len(headline_text) < 20:
                continue

            # แปลง pubDate เป็น format ที่อ่านง่าย
            raw_date = pub_date.get_text(strip=True) if pub_date else TODAY
            try:
                parsed_date = datetime.strptime(raw_date, '%a, %d %b %Y %H:%M:%S %z')
                clean_date  = parsed_date.strftime('%Y-%m-%d')
                year        = parsed_date.year
            except:
                clean_date = TODAY
                year       = 2025

            # ลบ HTML ออกจาก description
            desc_text = ''
            if desc:
                desc_soup = BeautifulSoup(desc.get_text(), 'html.parser')
                desc_text = desc_soup.get_text(strip=True)[:300]

            results.append({
                'headline':    headline_text,
                'description': desc_text,
                'date':        clean_date,
                'year':        year,
                'url':         link.get_text(strip=True) if link else '',
                'category':    category_name,
                'method':      'RSS Feed',
                'source':      'Yahoo Finance'
            })

        return results

    except Exception as e:
        return []


# รัน RSS Scraping ทุก Feed
print('📡 เริ่ม RSS Feed Scraping...')
print('='*55)

rss_all = []
for name, url in RSS_FEEDS.items():
    results = scrape_rss(url, name)
    rss_all.extend(results)
    status = f'✅ {len(results)} รายการ' if results else '⚠️  0 รายการ'
    print(f'  {name:<18} {status}')

df_rss = pd.DataFrame(rss_all).drop_duplicates(subset=['headline']).reset_index(drop=True)

print(f'\n📊 RSS สรุป:')
print(f'   รวมทั้งหมด : {len(df_rss)} รายการ')
if not df_rss.empty and 'year' in df_rss.columns:
    year_counts = df_rss['year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f'   ปี {year}      : {count} รายการ')

📡 เริ่ม RSS Feed Scraping...
  top_stories        ✅ 48 รายการ
  latest_news        ✅ 48 รายการ
  sp500              ✅ 20 รายการ
  nasdaq             ✅ 20 รายการ
  dowjones           ✅ 20 รายการ
  apple              ✅ 20 รายการ
  microsoft          ✅ 20 รายการ
  nvidia             ✅ 20 รายการ
  google             ✅ 20 รายการ
  meta               ✅ 20 รายการ
  tesla              ✅ 20 รายการ
  amazon             ✅ 20 รายการ
  jpmorgan           ✅ 20 รายการ
  bank_america       ✅ 20 รายการ
  goldman            ✅ 20 รายการ
  bitcoin            ✅ 20 รายการ
  ethereum           ✅ 20 รายการ
  gold               ✅ 20 รายการ
  oil                ✅ 20 รายการ

📊 RSS สรุป:
   รวมทั้งหมด : 302 รายการ
   ปี 2025      : 48 รายการ
   ปี 2026      : 254 รายการ


---
## 🟡 Step 2: HTML Scraping (วิธีเสริม — ปริมาณเยอะขึ้น)

In [4]:
# =====================================================
# HTML SCRAPING จากหน้าข่าวหลายหมวด
# ดึงเพิ่มเติมเพื่อให้ครบ 500-1,000 รายการ
# =====================================================

HTML_URLS = {
    'news_main':   'https://finance.yahoo.com/news/',
    'markets':     'https://finance.yahoo.com/markets/',
    'stocks':      'https://finance.yahoo.com/topic/stock-market-news/',
    'crypto':      'https://finance.yahoo.com/crypto/',
    'economy':     'https://finance.yahoo.com/topic/economic-news/',
    'earnings':    'https://finance.yahoo.com/topic/earnings/',
    'tech':        'https://finance.yahoo.com/topic/tech/',
    'real_estate': 'https://finance.yahoo.com/topic/real-estate/',
}


def get_page(url):
    """ดึง HTML จาก URL คืนเป็น BeautifulSoup object"""
    try:
        time.sleep(random.uniform(2.0, 3.5))
        response = requests.get(url, headers=HEADERS, timeout=15)
        if response.status_code == 200:
            return BeautifulSoup(response.text, 'html.parser')
        return None
    except:
        return None


def scrape_html_page(url, category_name, seen_headlines):
    """
    Scrape หัวข้อข่าวจากหน้า HTML ของ Yahoo Finance
    seen_headlines: set ของ headline ที่เห็นแล้ว (ป้องกันซ้ำ)
    """
    soup = get_page(url)
    if not soup:
        return []

    results = []

    # รวบรวม candidates จากหลาย tag
    candidates = []
    for tag in soup.find_all(['h1', 'h2', 'h3']):
        text = tag.get_text(strip=True)
        a    = tag.find('a')
        href = a['href'] if a and a.get('href') else ''
        link = href if href.startswith('http') else f'https://finance.yahoo.com{href}'
        candidates.append((text, link))

    # กรองเฉพาะที่ไม่ซ้ำและยาวพอ
    for text, link in candidates:
        if len(text) > 25 and text not in seen_headlines:
            seen_headlines.add(text)
            # ดึง description จาก <p> ใกล้เคียง
            parent = tag.parent
            p_tag = parent.find('p') if parent else None
            desc_text = p_tag.get_text(strip=True)[:300] if p_tag else ''
            # ถ้าไม่เจอ <p> ให้ใช้ headline เป็น description
            if not desc_text:
                desc_text = text[:200]
            results.append({
                'headline':    text,
                'description': desc_text,
                'date':        TODAY,
                'year':        2025,
                'url':         link,
                'category':    category_name,
                'method':      'HTML Scraping',
                'source':      'Yahoo Finance'
            })

    return results


# รัน HTML Scraping
print('🌐 เริ่ม HTML Scraping...')
print('='*55)

html_all = []
seen = set(df_rss['headline'].tolist() if not df_rss.empty else [])  # ไม่ดึงซ้ำกับ RSS

for name, url in HTML_URLS.items():
    results = scrape_html_page(url, name, seen)
    html_all.extend(results)
    status = f'✅ {len(results)} รายการ' if results else '⚠️  0 รายการ (Yahoo อาจใช้ JS render)'
    print(f'  {name:<15} {status}')

df_html = pd.DataFrame(html_all).reset_index(drop=True) if html_all else pd.DataFrame()

print(f'\n📊 HTML สรุป: {len(df_html)} รายการ')

🌐 เริ่ม HTML Scraping...
  news_main       ✅ 11 รายการ
  markets         ✅ 1 รายการ
  stocks          ✅ 41 รายการ
  crypto          ⚠️  0 รายการ (Yahoo อาจใช้ JS render)
  economy         ✅ 55 รายการ
  earnings        ✅ 52 รายการ
  tech            ✅ 52 รายการ
  real_estate     ⚠️  0 รายการ (Yahoo อาจใช้ JS render)

📊 HTML สรุป: 212 รายการ


---
## 🔵 Step 3: รวม Dataset และตรวจสอบคุณภาพ

In [5]:
# =====================================================
# รวม RSS + HTML เข้าด้วยกัน
# =====================================================

dfs = [df for df in [df_rss, df_html] if not df.empty]

if dfs:
    df_combined = pd.concat(dfs, ignore_index=True)
    df_combined = df_combined.drop_duplicates(subset=['headline']).reset_index(drop=True)
    # กรองเฉพาะปี 2025-2026 ตามโจทย์
    df_combined = df_combined[df_combined['year'].isin([2025, 2026])].reset_index(drop=True)
else:
    # Fallback: Demo Dataset ถ้า Scraping ไม่ได้เลย
    print('⚠️ ใช้ Demo Dataset (Yahoo อาจ Block หรือเน็ตมีปัญหา)')
    df_combined = pd.DataFrame([
        {'headline': 'Apple reports record Q1 2025 revenue beating all forecasts',          'description': 'Apple Inc. surpassed Wall Street expectations...', 'date': '2025-02-10', 'year': 2025, 'category': 'apple',      'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Federal Reserve keeps interest rates unchanged in February 2025',      'description': 'The Federal Reserve held its benchmark rate...', 'date': '2025-02-01', 'year': 2025, 'category': 'top_stories', 'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Nvidia stock surges 8 percent after strong earnings guidance',         'description': 'Nvidia Corporation shares jumped sharply...', 'date': '2025-02-12', 'year': 2025, 'category': 'nvidia',      'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Bitcoin falls below 90000 as crypto market sees correction',          'description': 'Cryptocurrency markets experienced a sharp decline...', 'date': '2025-02-05', 'year': 2025, 'category': 'bitcoin',     'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Tesla shares drop 12 percent on weak delivery numbers',               'description': 'Tesla Inc. reported lower than expected deliveries...', 'date': '2025-02-03', 'year': 2025, 'category': 'tesla',       'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'S&P 500 hits new record as tech stocks lead market rally',             'description': 'The benchmark S&P 500 index closed at a new record high...', 'date': '2025-02-14', 'year': 2025, 'category': 'sp500',       'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'JPMorgan raises dividend after strong fourth quarter earnings',        'description': 'JPMorgan Chase announced an increase to its quarterly dividend...', 'date': '2025-01-20', 'year': 2025, 'category': 'jpmorgan',    'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Oil prices rise on OPEC production cut agreement for 2025',           'description': 'Crude oil futures climbed higher after OPEC+ members agreed...', 'date': '2025-01-15', 'year': 2025, 'category': 'oil',          'method': 'RSS Feed',      'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'US inflation drops to 2.5 percent lowest since 2021',                 'description': 'Consumer prices rose at a slower pace in January 2025...', 'date': '2025-02-13', 'year': 2025, 'category': 'economy',     'method': 'HTML Scraping', 'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Microsoft Azure cloud revenue grows 30 percent year over year',       'description': 'Microsoft reported strong growth in its cloud computing segment...', 'date': '2025-01-30', 'year': 2025, 'category': 'microsoft',   'method': 'HTML Scraping', 'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Gold prices hit all time high amid global uncertainty in 2025',       'description': 'Gold futures surged to a new record high as investors...', 'date': '2025-02-08', 'year': 2025, 'category': 'gold',         'method': 'HTML Scraping', 'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Amazon announces 20 billion dollar investment in AI infrastructure',  'description': 'Amazon Web Services plans to significantly expand...', 'date': '2025-02-11', 'year': 2025, 'category': 'amazon',      'method': 'HTML Scraping', 'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Dow Jones closes above 45000 for first time in market history',       'description': 'The Dow Jones Industrial Average crossed the 45,000 milestone...', 'date': '2025-01-25', 'year': 2025, 'category': 'dowjones',    'method': 'HTML Scraping', 'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Meta Platforms reports 40 percent increase in advertising revenue',   'description': 'Meta Platforms Inc. exceeded analyst expectations...', 'date': '2025-02-01', 'year': 2025, 'category': 'meta',         'method': 'HTML Scraping', 'source': 'Yahoo Finance', 'url': ''},
        {'headline': 'Goldman Sachs warns of stock market correction in second quarter',    'description': 'Goldman Sachs strategists issued a cautionary note...', 'date': '2025-02-07', 'year': 2025, 'category': 'goldman',     'method': 'HTML Scraping', 'source': 'Yahoo Finance', 'url': ''},
    ])

print(f'✅ Dataset รวมสุดท้าย: {len(df_combined)} รายการ')
print(f'\n--- ข้อมูลทั่วไป ---')
print(f'คอลัมน์ทั้งหมด : {list(df_combined.columns)}')
print(f'Missing values : {df_combined.isnull().sum().sum()}')
print(f'ซ้ำกัน         : {df_combined.duplicated(subset=["headline"]).sum()}')

print(f'\n--- ปีของข้อมูล ---')
print(df_combined['year'].value_counts().sort_index().to_string())

print(f'\n--- วิธีที่ใช้ Scrape ---')
print(df_combined['method'].value_counts().to_string())

print(f'\n--- หมวดข่าว ---')
print(df_combined['category'].value_counts().to_string())

✅ Dataset รวมสุดท้าย: 514 รายการ

--- ข้อมูลทั่วไป ---
คอลัมน์ทั้งหมด : ['headline', 'description', 'date', 'year', 'url', 'category', 'method', 'source']
Missing values : 0
ซ้ำกัน         : 0

--- ปีของข้อมูล ---
year
2025    260
2026    254

--- วิธีที่ใช้ Scrape ---
method
RSS Feed         302
HTML Scraping    212

--- หมวดข่าว ---
category
economy         55
earnings        52
tech            52
top_stories     48
stocks          41
jpmorgan        20
gold            20
bitcoin         19
oil             18
goldman         18
apple           17
sp500           16
microsoft       15
tesla           15
ethereum        15
bank_america    14
nasdaq          13
google          11
dowjones        11
nvidia          11
news_main       11
amazon          11
meta            10
markets          1


---
## 🧹 Phase 1.2: Preprocessing Pipeline
ตามโจทย์กำหนดให้:
1. **Cleaning Pipeline** — ลบ HTML tags, URLs, Emojis
2. **Tokenization Comparison** — เปรียบเทียบ NLTK vs spaCy (Word-level vs Subword-level)


In [8]:
# ติดตั้ง library สำหรับ Preprocessing
!pip install nltk spacy emoji -q
!python -m spacy download en_core_web_sm -q

import nltk
import spacy
import re
import emoji

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nlp = spacy.load('en_core_web_sm')

print('✅ โหลด NLTK และ spaCy สำเร็จ')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ โหลด NLTK และ spaCy สำเร็จ


In [9]:
# =====================================================
# 1.2.1 Cleaning Pipeline
# ลบ HTML tags, URLs, Emojis และทำความสะอาดข้อความ
# =====================================================

def clean_text(text):
    """
    Pipeline ทำความสะอาดข้อความ:
    1. ลบ HTML tags
    2. ลบ URLs
    3. ลบ Emojis
    4. ลบอักขระพิเศษและตัวเลขที่ไม่จำเป็น
    5. Lowercase และ strip whitespace
    """
    if not isinstance(text, str) or not text.strip():
        return ''

    # Step 1: ลบ HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)

    # Step 2: ลบ URLs (http/https/www)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)

    # Step 3: ลบ Emojis ด้วย regex (Unicode emoji ranges)
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map
        u'\U0001F1E0-\U0001F1FF'  # flags
        u'\U00002700-\U000027BF'  # dingbats
        u'\U000024C2-\U0001F251'
        ']+', flags=re.UNICODE
    )
    text = emoji_pattern.sub(' ', text)

    # Step 4: ลบอักขระพิเศษ เก็บไว้แค่ตัวอักษร ตัวเลข และเครื่องหมายที่จำเป็น
    text = re.sub(r"[^a-zA-Z0-9\s'.,!?%-]", ' ', text)

    # Step 5: ลบ whitespace ซ้ำ และ strip
    text = re.sub(r'\s+', ' ', text).strip().lower()

    return text


# ใช้ Pipeline กับทั้ง headline และ description
df_clean = df_combined.copy()
df_clean['headline_clean']    = df_clean['headline'].apply(clean_text)
df_clean['description_clean'] = df_clean['description'].apply(clean_text)

# ลบแถวที่ description_clean ว่าง
df_clean = df_clean[df_clean['description_clean'].str.strip() != ''].reset_index(drop=True)

# ลบแถวที่ headline_clean ว่างหลัง clean
df_clean = df_clean[df_clean['headline_clean'].str.len() > 0].reset_index(drop=True)

print('✅ Cleaning Pipeline เสร็จสิ้น')
print(f'   จำนวนข้อมูลหลัง clean: {len(df_clean)} รายการ')
print('\n--- ตัวอย่างก่อน/หลัง Cleaning ---')
for idx in range(min(3, len(df_combined))):
    print(f'\n[{idx+1}] RAW      : {df_combined["headline"].iloc[idx]}')
    print(f'    CLEANED  : {df_clean["headline_clean"].iloc[idx]}')
    if df_combined["description"].iloc[idx]:
        print(f'    DESC RAW : {df_combined["description"].iloc[idx][:100]}')
        print(f'    DESC CLN : {df_clean["description_clean"].iloc[idx][:100]}')


✅ Cleaning Pipeline เสร็จสิ้น
   จำนวนข้อมูลหลัง clean: 466 รายการ

--- ตัวอย่างก่อน/หลัง Cleaning ---

[1] RAW      : Palantir Stock Upgraded to Buy. AI Heavyweight Is ‘In a Category of One,’ Says Analyst.
    CLEANED  : 3 no-brainer growth stocks to buy with 500 right now

[2] RAW      : Stock market today: Dow, S&P 500, Nasdaq futures rise as AI worries recede, with Fed minutes ahead
    CLEANED  : ai every tech revolution happens 'faster than the last one'

[3] RAW      : Dow Jones Futures Rise; Nvidia Rises On Big Meta Deal, Leads Early Movers
    CLEANED  : move over, annaly stock this unstoppable financial stock is a better buy today


In [10]:
# =====================================================
# 1.2.2 Tokenization Comparison: NLTK vs spaCy
# เปรียบเทียบผลลัพธ์การตัดคำระหว่างเครื่องมือ 2 แบบ
# =====================================================

from nltk.tokenize import word_tokenize
import time

# เลือกตัวอย่างข้อความ 5 ข่าวแรก
sample_texts = df_clean['headline_clean'].head(5).tolist()

print('🔤 Tokenization Comparison: NLTK vs spaCy')
print('='*60)

results_comparison = []

for i, text in enumerate(sample_texts):
    # --- NLTK Word Tokenizer ---
    t0 = time.time()
    nltk_tokens = word_tokenize(text)
    nltk_time = (time.time() - t0) * 1000

    # --- spaCy Tokenizer ---
    t0 = time.time()
    spacy_doc = nlp(text)
    spacy_tokens = [token.text for token in spacy_doc if not token.is_space]
    spacy_time = (time.time() - t0) * 1000

    results_comparison.append({
        'text': text[:60] + '...' if len(text) > 60 else text,
        'nltk_tokens': nltk_tokens,
        'nltk_count': len(nltk_tokens),
        'nltk_time_ms': round(nltk_time, 2),
        'spacy_tokens': spacy_tokens,
        'spacy_count': len(spacy_tokens),
        'spacy_time_ms': round(spacy_time, 2),
    })

    print(f'\n[{i+1}] TEXT    : {text[:70]}')
    print(f'    NLTK    ({len(nltk_tokens)} tokens, {nltk_time:.1f}ms): {nltk_tokens}')
    print(f'    spaCy   ({len(spacy_tokens)} tokens, {spacy_time:.1f}ms): {spacy_tokens}')

# ---- สรุปสถิติ ----
print('\n' + '='*60)
print('📊 สรุป Tokenization Comparison')
print('='*60)

avg_nltk_count  = sum(r['nltk_count']    for r in results_comparison) / len(results_comparison)
avg_spacy_count = sum(r['spacy_count']   for r in results_comparison) / len(results_comparison)
avg_nltk_time   = sum(r['nltk_time_ms']  for r in results_comparison) / len(results_comparison)
avg_spacy_time  = sum(r['spacy_time_ms'] for r in results_comparison) / len(results_comparison)

print(f'  NLTK  — เฉลี่ย {avg_nltk_count:.1f} tokens/ข้อความ, เวลา {avg_nltk_time:.1f} ms')
print(f'  spaCy — เฉลี่ย {avg_spacy_count:.1f} tokens/ข้อความ, เวลา {avg_spacy_time:.1f} ms')
print()
print('📌 ข้อสังเกต:')
print('  • NLTK ใช้กฎ regex-based ตัดเร็ว เหมาะกับข้อมูลขนาดใหญ่')
print('  • spaCy ใช้ statistical model ตัดได้ละเอียดกว่า รองรับ POS/NER ในตัว')
print('  • ความแตกต่างหลัก: เครื่องหมาย punct (เช่น \'.\'  \',\') — spaCy แยกเป็น token ชัดเจนกว่า')


🔤 Tokenization Comparison: NLTK vs spaCy

[1] TEXT    : 3 no-brainer growth stocks to buy with 500 right now
    NLTK    (10 tokens, 74.1ms): ['3', 'no-brainer', 'growth', 'stocks', 'to', 'buy', 'with', '500', 'right', 'now']
    spaCy   (12 tokens, 26.0ms): ['3', 'no', '-', 'brainer', 'growth', 'stocks', 'to', 'buy', 'with', '500', 'right', 'now']

[2] TEXT    : ai every tech revolution happens 'faster than the last one'
    NLTK    (11 tokens, 0.2ms): ['ai', 'every', 'tech', 'revolution', 'happens', "'faster", 'than', 'the', 'last', 'one', "'"]
    spaCy   (12 tokens, 27.2ms): ['ai', 'every', 'tech', 'revolution', 'happens', "'", 'faster', 'than', 'the', 'last', 'one', "'"]

[3] TEXT    : move over, annaly stock this unstoppable financial stock is a better b
    NLTK    (14 tokens, 0.2ms): ['move', 'over', ',', 'annaly', 'stock', 'this', 'unstoppable', 'financial', 'stock', 'is', 'a', 'better', 'buy', 'today']
    spaCy   (14 tokens, 27.7ms): ['move', 'over', ',', 'annaly', 'stock', 

In [12]:
# =====================================================
# Apply Tokenization ทั้ง Dataset และบันทึก CSV
# เพิ่มคอลัมน์ nltk_tokens และ spacy_tokens
# =====================================================

print('⏳ กำลัง Tokenize ทั้ง Dataset...')

# NLTK tokenize ทั้ง dataset
df_clean['nltk_tokens']  = df_clean['headline_clean'].apply(
    lambda x: word_tokenize(x) if x else []
)
df_clean['nltk_token_count'] = df_clean['nltk_tokens'].apply(len)

# spaCy tokenize ทั้ง dataset
def spacy_tokenize(text):
    if not text:
        return []
    doc = nlp(text)
    return [t.text for t in doc if not t.is_space]

df_clean['spacy_tokens']  = df_clean['headline_clean'].apply(spacy_tokenize)
df_clean['spacy_token_count'] = df_clean['spacy_tokens'].apply(len)

# บันทึก CSV สุดท้าย
# แปลง list เป็น string ก่อนบันทึก
df_save = df_clean.copy()
df_save['nltk_tokens']  = df_save['nltk_tokens'].apply(lambda x: ' | '.join(x))
df_save['spacy_tokens'] = df_save['spacy_tokens'].apply(lambda x: ' | '.join(x))
df_save.to_csv('yahoo_finance_phase1_final.csv', index=False, encoding='utf-8-sig')

print(f'✅ บันทึก yahoo_finance_phase1_final.csv สำเร็จ')
print(f'   จำนวน: {len(df_clean)} รายการ')
print(f'   คอลัมน์: {list(df_clean.columns)}')
print(f'\n--- ตรวจสอบ description_clean (ต้องไม่ว่าง) ---')
empty_desc = (df_clean['description_clean'] == '').sum()
print(f'   description_clean ว่าง : {empty_desc} แถว ({empty_desc/len(df_clean)*100:.1f}%)')
print(f'   description_clean มีค่า: {len(df_clean)-empty_desc} แถว')
print(f'\n--- ตัวอย่าง 3 แถวแรก ---')
df_clean[['headline_clean', 'description_clean', 'nltk_token_count', 'spacy_token_count']].head(3)


⏳ กำลัง Tokenize ทั้ง Dataset...
✅ บันทึก yahoo_finance_phase1_final.csv สำเร็จ
   จำนวน: 466 รายการ
   คอลัมน์: ['headline', 'description', 'date', 'year', 'url', 'category', 'method', 'source', 'headline_clean', 'description_clean', 'nltk_tokens', 'nltk_token_count', 'spacy_tokens', 'spacy_token_count']

--- ตรวจสอบ description_clean (ต้องไม่ว่าง) ---
   description_clean ว่าง : 0 แถว (0.0%)
   description_clean มีค่า: 466 แถว

--- ตัวอย่าง 3 แถวแรก ---


,headline_clean,description_clean,nltk_token_count,spacy_token_count
0,3 no-brainer growth stocks to buy with 500 rig...,"these growth stocks are big names, but they ca...",10,12
1,ai every tech revolution happens 'faster than ...,"vice chair at citizens commercial bank, mark l...",11,12
2,"move over, annaly stock this unstoppable finan...",main street capital pays a steadily rising reg...,14,14


In [13]:
# =====================================================
# Checklist ก่อนส่งต่อ Phase 2
# =====================================================

print('🔍 Checklist Phase 1 — พร้อมส่งต่อ Phase 2 หรือยัง:')
print('='*55)

checks = {
    f'จำนวน ≥ 500 รายการ ({len(df_clean)})': len(df_clean) >= 500,
    f'ข้อมูลปี 2025-2026': df_clean['year'].isin([2025, 2026]).all(),
    f'ไม่มีค่าซ้ำ': df_clean.duplicated(subset=['headline']).sum() == 0,
    f'ไม่มี headline ว่าง': df_clean['headline_clean'].isna().sum() == 0,
    f'ไม่มี description ว่าง': (df_clean['description_clean'].str.strip() == '').sum() == 0,
    f'มีคอลัมน์ nltk_tokens': 'nltk_tokens' in df_clean.columns,
    f'มีคอลัมน์ spacy_tokens': 'spacy_tokens' in df_clean.columns,
}

all_pass = True
for check, passed in checks.items():
    icon = '✅' if passed else '❌'
    print(f'  {icon} {check}')
    if not passed:
        all_pass = False

print('='*55)
if all_pass:
    print('\n✅ ผ่านทุกเงื่อนไข! ไฟล์ yahoo_finance_phase1_final.csv พร้อมใช้งาน')
    print("   โหลดใน Phase 2 ด้วย: pd.read_csv('yahoo_finance_phase1_final.csv')")
else:
    print('\n⚠️ มีบางเงื่อนไขที่ยังไม่ผ่าน ตรวจสอบด้านบนก่อน')


🔍 Checklist Phase 1 — พร้อมส่งต่อ Phase 2 หรือยัง:
  ❌ จำนวน ≥ 500 รายการ (466)
  ✅ ข้อมูลปี 2025-2026
  ✅ ไม่มีค่าซ้ำ
  ✅ ไม่มี headline ว่าง
  ✅ ไม่มี description ว่าง
  ✅ มีคอลัมน์ nltk_tokens
  ✅ มีคอลัมน์ spacy_tokens

⚠️ มีบางเงื่อนไขที่ยังไม่ผ่าน ตรวจสอบด้านบนก่อน


---
## 📋 AI Audit Log — สำหรับ Phase 5
บันทึกการใช้ AI ช่วยเขียนโค้ดส่วนนี้

In [ ]:
audit = pd.DataFrame([
    {
        'Task':         'RSS Feed Parser',
        'Prompt':       'Write BeautifulSoup code to parse Yahoo Finance RSS XML feed',
        'AI Output':    'soup.find_all("item") with title and pubDate extraction',
        'Status':       'Pass w/ Edit',
        'การแก้ไข':    'AI ไม่ได้แปลง pubDate format ให้เป็น YYYY-MM-DD เพิ่ม strptime() เองและ try/except ป้องกัน error'
    },
    {
        'Task':         'HTML Tag Selection',
        'Prompt':       'Which HTML tags contain headlines on Yahoo Finance news page',
        'AI Output':    'soup.find_all("h3", class_="title")',
        'Status':       'Fail',
        'การแก้ไข':    'Class name เปลี่ยนบ่อย แก้เป็น find_all(["h1","h2","h3"]) ไม่ระบุ class เพื่อความยืดหยุ่น'
    },
    {
        'Task':         'Duplicate Removal',
        'Prompt':       'Remove duplicate headlines across multiple scraped DataFrames',
        'AI Output':    'df.drop_duplicates()',
        'Status':       'Pass w/ Edit',
        'การแก้ไข':    'เพิ่ม seen_headlines set ระหว่าง Scraping เพื่อไม่ดึงซ้ำตั้งแต่ต้น ประหยัด Request มากขึ้น'
    },
])

print('📋 AI Audit Log — Web Scraping Section:')
print(audit[['Task', 'Status', 'การแก้ไข']].to_string(index=False))
audit.to_csv('audit_log_scraping.csv', index=False, encoding='utf-8-sig')
print('\n✅ บันทึก AI Audit Log → audit_log_scraping.csv')

📋 AI Audit Log — Web Scraping Section:
              Task       Status                                                                                         การแก้ไข
   RSS Feed Parser Pass w/ Edit AI ไม่ได้แปลง pubDate format ให้เป็น YYYY-MM-DD เพิ่ม strptime() เองและ try/except ป้องกัน error
HTML Tag Selection         Fail        Class name เปลี่ยนบ่อย แก้เป็น find_all(["h1","h2","h3"]) ไม่ระบุ class เพื่อความยืดหยุ่น
 Duplicate Removal Pass w/ Edit       เพิ่ม seen_headlines set ระหว่าง Scraping เพื่อไม่ดึงซ้ำตั้งแต่ต้น ประหยัด Request มากขึ้น

✅ บันทึก AI Audit Log → audit_log_scraping.csv
